In [1]:
!cp /kaggle/input/datasets-wheel/datasets-2.14.4-py3-none-any.whl /kaggle/working
!pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl

Processing ./datasets-2.14.4-py3-none-any.whl
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [2]:
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing ./sentence-transformers
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126125 sha256=fec9850782ee63cc694e65e2d5f8f60e36768077c5b99a035543f9dca6b3cde0
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205b854faf1812
Successfully built sentence-transformers
Processing /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl
Processing /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0
Processing /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
Processing /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl


In [3]:
import numpy as np
import pandas as pd 
from datasets import load_dataset, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import LongformerTokenizer, LongformerForMultipleChoice
import transformers
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import unicodedata

import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
!cp -r /kaggle/input/stem-wiki-cohere-no-emb /kaggle/working
!cp -r /kaggle/input/all-paraphs-parsed-expanded /kaggle/working/

In [5]:
def SplitList(mylist, chunk_size):
    return [mylist[offs:offs+chunk_size] for offs in range(0, len(mylist), chunk_size)]
def get_relevant_documents_parsed(df_valid):
    df_chunk_size=500
    paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
    modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ").replace("'","")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         paraphs_parsed_dataset[idx.item()]["title"],
                         paraphs_parsed_dataset[idx.item()]["text"],
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles
def get_relevant_documents(df_valid):
    df_chunk_size=700
    
    cohere_dataset_filtered = load_from_disk("/kaggle/working/stem-wiki-cohere-no-emb")
    modified_texts = cohere_dataset_filtered.map(lambda example:
                                             {'temp_text':
                                              unicodedata.normalize("NFKD", f"{example['title']} {example['text']}").replace('"',"")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         cohere_dataset_filtered[idx.item()]["title"],
                         unicodedata.normalize("NFKD", cohere_dataset_filtered[idx.item()]["text"]),
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles
def retrieval(df_valid, modified_texts):
    
    corpus_df_valid = df_valid.apply(lambda row:
                                     f'{row["prompt"]}\n{row["prompt"]}\n{row["prompt"]}\n{row["A"]}\n{row["B"]}\n{row["C"]}\n{row["D"]}\n{row["E"]}',
                                     axis=1).values
    vectorizer1 = TfidfVectorizer(ngram_range=(1,3),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words,
                                 strip_accents="unicode",sublinear_tf=True)
    vectorizer1.fit(corpus_df_valid)
    vocab_df_valid = vectorizer1.get_feature_names_out()
    vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words,
                                 vocabulary=vocab_df_valid,
                                strip_accents="unicode",sublinear_tf=True)
    vectorizer.fit(modified_texts[:500000])
    corpus_tf_idf = vectorizer.transform(corpus_df_valid)
    
#     vectorizer = BM25()
#     vectorizer.fit(corpus_df_valid)
#     corpus_tf_idf = vectorizer.transform(corpus_df_valid)
    print(f"length of vectorizer vocab is {len(vectorizer.get_feature_names_out())}")

    chunk_size = 100000
    top_per_chunk = 10
    top_per_query = 10

    all_chunk_top_indices = []
    all_chunk_top_values = []

    for idx in tqdm(range(0, len(modified_texts), chunk_size)):
        wiki_vectors = vectorizer.transform(modified_texts[idx: idx+chunk_size])
        temp_scores = (corpus_tf_idf * wiki_vectors.T).toarray()
        chunk_top_indices = temp_scores.argpartition(-top_per_chunk, axis=1)[:, -top_per_chunk:]
        chunk_top_values = temp_scores[np.arange(temp_scores.shape[0])[:, np.newaxis], chunk_top_indices]

        all_chunk_top_indices.append(chunk_top_indices + idx)
        all_chunk_top_values.append(chunk_top_values)

    top_indices_array = np.concatenate(all_chunk_top_indices, axis=1)
    top_values_array = np.concatenate(all_chunk_top_values, axis=1)
    
    merged_top_scores = np.sort(top_values_array, axis=1)[:,-top_per_query:]
    merged_top_indices = top_values_array.argsort(axis=1)[:,-top_per_query:]
    articles_indices = top_indices_array[np.arange(top_indices_array.shape[0])[:, np.newaxis], merged_top_indices]
    
    return articles_indices, merged_top_scores
def prepare_answering_input(
        tokenizer, 
        question,  
        options,   
        context,   
        max_seq_length=700,
        ):
        c_plus_q   = context + '\n' + tokenizer.bos_token + '\n' + question
        c_plus_q_4 = [c_plus_q] * len(options)
        tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,
        padding="longest",
        truncation=False,
        return_tensors="pt",
        )
        input_ids = tokenized_examples['input_ids'].unsqueeze(0)
        attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)
        example_encoded = {
            "input_ids": input_ids.to(model.device.index),
            "attention_mask": attention_mask.to(model.device.index),
        }
        return example_encoded

In [6]:
stop_words = ['each', 'you', 'the', 'use', 'used',
                  'where', 'themselves', 'nor', "it's", 'how', "don't", 'just', 'your',
                  'about', 'himself', 'with', "weren't", 'hers', "wouldn't", 'more', 'its', 'were',
                  'his', 'their', 'then', 'been', 'myself', 're', 'not',
                  'ours', 'will', 'needn', 'which', 'here', 'hadn', 'it', 'our', 'there', 'than',
                  'most', "couldn't", 'both', 'some', 'for', 'up', 'couldn', "that'll",
                  "she's", 'over', 'this', 'now', 'until', 'these', 'few', 'haven',
                  'of', 'wouldn', 'into', 'too', 'to', 'very', 'shan', 'before', 'the', 'they',
                  'between', "doesn't", 'are', 'was', 'out', 'we', 'me',
                  'after', 'has', "isn't", 'have', 'such', 'should', 'yourselves', 'or', 'during', 'herself',
                  'doing', 'in', "shouldn't", "won't", 'when', 'do', 'through', 'she',
                  'having', 'him', "haven't", 'against', 'itself', 'that',
                  'did', 'theirs', 'can', 'those',
                  'own', 'so', 'and', 'who', "you've", 'yourself', 'her', 'he', 'only',
                  'what', 'ourselves', 'again', 'had', "you'd", 'is', 'other',
                  'why', 'while', 'from', 'them', 'if', 'above', 'does', 'whom',
                  'yours', 'but', 'being', "wasn't", 'be']
from sklearn.feature_extraction import text
stop_words2 = text.ENGLISH_STOP_WORDS
import spacy
nlp = spacy.load("en_core_web_sm")
stop_words3 = set(nlp.Defaults.stop_words)
from gensim.parsing.preprocessing import STOPWORDS
stop_words4 = set(STOPWORDS)
stop_words = list(stop_words2.union(stop_words,stop_words3,stop_words4))

In [7]:
df_valid = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")

In [8]:
import gc

In [9]:
retrieved_articles_parsed = get_relevant_documents_parsed(df_valid)
gc.collect()

Map (num_proc=2):   0%|          | 0/2101279 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'd', 'isn', 'll', 'm', 'n', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won'] not in stop_words.
  warnings.warn(


length of vectorizer vocab is 19346



100%|██████████| 1/1 [07:44<00:00, 464.91s/it]


298

In [10]:
retrieved_articles = get_relevant_documents(df_valid)
gc.collect()

Map (num_proc=2):   0%|          | 0/2781652 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

length of vectorizer vocab is 19346



100%|██████████| 1/1 [05:39<00:00, 339.27s/it]


18

In [11]:
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

In [12]:
model_dir = "/kaggle/input/llm-science-run-context-2"
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [13]:
model_ckpts = [
    "/kaggle/input/model_1",
    "/kaggle/input/model_2",

    "/kaggle/input/model_3",
    "/kaggle/input/model_4"
]

In [14]:
weights=[0.21417155322179493, 0.22915370782572908, 0.26221393214844196, 0.29446080680403397]

In [15]:
w=[]
g=0
for i in model_ckpts:
    print(i,':',weights[g])
    g+=1
    predictions = []
    submit_ids = []
    x=[]
    model = AutoModelForMultipleChoice.from_pretrained(i).cuda()
    tokenizer = AutoTokenizer.from_pretrained(i)
    for index in tqdm(range(df_valid.shape[0])):
        columns = df_valid.iloc[index].values
        submit_ids.append(columns[0])
        question ="Answer the following questions by eliminating wrong options:\n"+ columns[1]
        options = [columns[2], columns[3], columns[4], columns[5], columns[6]]
        context1 = f"{retrieved_articles[index][-5][2]}\n{retrieved_articles[index][-4][2]}\n{retrieved_articles[index][-3][2]}\n{retrieved_articles[index][-2][2]}\n{retrieved_articles[index][-1][2]}\n{retrieved_articles[index][0][2]}"
        context2 = f"{retrieved_articles_parsed[index][-3][2]}\n{retrieved_articles_parsed[index][-2][2]}\n{retrieved_articles_parsed[index][-1][2]}"
        inputs1 = prepare_answering_input(
            tokenizer=tokenizer, question=question,
            options=options, context=context1,
            )
        inputs2 = prepare_answering_input(
            tokenizer=tokenizer, question=question,
            options=options, context=context2,
            )
    
        with torch.no_grad():
            outputs1 = model(**inputs1)    
            losses1 = -outputs1.logits[0].detach().cpu().numpy()
            probability1 = torch.softmax(torch.tensor(-losses1), dim=-1)
        
        with torch.no_grad():
            outputs2 = model(**inputs2)
            losses2 = -outputs2.logits[0].detach().cpu().numpy()
            probability2 = torch.softmax(torch.tensor(-losses2), dim=-1)
        
        probability_ = (probability1 + probability2)/2
        x.append(probability_)
    w.append(x)



/kaggle/input/how-to-use-40k-dataset/model_v3 : 0.21417155322179493


100%|██████████| 200/200 [03:23<00:00,  1.02s/it]


/kaggle/input/abcdefghijklmnopqrstuvwxyz : 0.22915370782572908


100%|██████████| 200/200 [03:22<00:00,  1.01s/it]


/kaggle/input/llm-science-exam-context-v2-models/deberta-checkpoint-7600/checkpoint-7600 : 0.26221393214844196


100%|██████████| 200/200 [03:22<00:00,  1.01s/it]


/kaggle/input/run-sci-1/model_v8 : 0.29446080680403397


100%|██████████| 200/200 [03:22<00:00,  1.01s/it]


In [16]:
predict=[]
for i in range(len(w[0])):
    predict+=[w[0][i]*weights[0]+w[1][i]*weights[1]+w[2][i]*weights[2]+w[3][i]*weights[3]]#+w[4][i]*weights[4]+w[5][i]*weights[5]]

In [17]:
for i in predict :
    predict = np.array(list("ABCDE"))[np.argsort(i)][-3:].tolist()[::-1]
    predictions.append(predict)
predictions = [" ".join(i) for i in predictions]

In [18]:
pd.DataFrame({'id':submit_ids,'prediction':predictions}).to_csv('submission.csv', index=False)

In [19]:
def calculate_MAP3(predictions, labels):
    U = len(predictions)  # Number of questions in the test set
    MAP3 = 0.0  # Mean Average Precision @ 3

    for i in range(U):
        n = len(predictions[i])  # Number of predictions per question
        relevant_labels = set(labels[i])  # Correct labels for the current question
        
        precision_sum = 0.0
        precision_at_k = 0.0
        relevant_count = 0

        for k in range(n):
            if predictions[i][k] in relevant_labels:
                relevant_count += 1
                precision_at_k = relevant_count / (k + 1)
                precision_sum += precision_at_k
                relevant_labels.remove(predictions[i][k])

            if relevant_count >= 3:
                break

        average_precision = precision_sum / min(len(labels[i]), 3)
        MAP3 += average_precision
    
    MAP3 /= U
    print("MAP@3 score:", MAP3)
from pathlib import Path
data_path = Path('/kaggle/input/kaggle-llm-science-exam')

training_set = pd.read_csv(data_path / "train.csv", index_col='id')
labels = [[x] for x in training_set['answer']]
calculate_MAP3(predictions, labels)

MAP@3 score: 0.9866666666666667


In [20]:
l=[]
for i in predictions:
    l.append(i[0])

import pandas as pd
l1=[]
tn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")
for i in tn['answer']:
    l1.append(i)
    #print(i)
sum=0
for i in range(len(l)):
    if l[i]==l1[i]:
        sum+=1
        
print(sum/len(l))

0.98
